# 06. Transformer Encoder Block 구현

## 학습 목표
- 지금까지 만든 컴포넌트 조립
- Residual Connection 이해
- Pre-LN vs Post-LN 구조

## 관련 면접 질문
- Encoder Block의 구조를 설명해주세요
- Residual Connection이 왜 필요한가요?
- BERT는 몇 개의 Encoder Block을 사용하나요?

---
## Pre-Quiz

### Q1. Encoder Block의 두 가지 sub-layer는?
```
1. 
2. 
```

### Q2. Residual Connection이 필요한 이유는?
```
당신의 답: 
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional

torch.manual_seed(42)

In [ ]:
# 이전에 구현한 컴포넌트들 (제공)
def scaled_dot_product_attention(query, key, value, mask=None):
    d_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask, float('-inf'))
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value), weights

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads, dropout=0.0):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)
        seq_q, seq_k = query.size(1), key.size(1)
        
        Q = self.W_q(query).view(batch_size, seq_q, self.num_heads, self.d_k).transpose(1, 2)
        K = self.W_k(key).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        V = self.W_v(value).view(batch_size, seq_k, self.num_heads, self.d_k).transpose(1, 2)
        
        Q = Q.contiguous().view(batch_size * self.num_heads, seq_q, self.d_k)
        K = K.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        V = V.contiguous().view(batch_size * self.num_heads, seq_k, self.d_k)
        
        output, weights = scaled_dot_product_attention(Q, K, V, mask)
        
        output = output.view(batch_size, self.num_heads, seq_q, self.d_k)
        output = output.transpose(1, 2).contiguous().view(batch_size, seq_q, self.d_model)
        
        return self.dropout(self.W_o(output)), weights.view(batch_size, self.num_heads, seq_q, seq_k)

class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff=None, dropout=0.1):
        super().__init__()
        if d_ff is None:
            d_ff = d_model * 4
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        return self.dropout(self.fc2(self.dropout(F.gelu(self.fc1(x)))))

---
## 1. Encoder Block 구현

### 구조 (Post-LN, 원래 Transformer)
```
x → MultiHeadAttention → + → LayerNorm → FFN → + → LayerNorm → output
    ↑___________________|                       ↑_________|
         (residual)                              (residual)
```

### 구조 (Pre-LN, 현대적)
```
x → LayerNorm → MultiHeadAttention → + → LayerNorm → FFN → + → output
                                     ↑                     ↑
    ↑________________________________|   ↑________________|
              (residual)                    (residual)
```

In [ ]:
# =================================================
# TODO: Encoder Block 구현 (Pre-LN 방식)
# =================================================
#
# 구조 (Pre-LN):
#   1. norm1 → attention → dropout → residual
#   2. norm2 → ffn → residual
#
# =================================================

class EncoderBlock(nn.Module):
    def __init__(
        self,
        d_model: int,
        num_heads: int,
        d_ff: int = None,
        dropout: float = 0.1
    ):
        super().__init__()
        
        # ============================================
        # 여기에 구현하세요
        # - attention: MultiHeadAttention
        # - ffn: PositionWiseFeedForward
        # - norm1, norm2: LayerNorm
        # - dropout
        # ============================================
        
        pass  # 이 줄을 지우고 구현
    
    def forward(
        self,
        x: torch.Tensor,
        mask: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        """
        Args:
            x: (batch, seq, d_model)
            mask: optional attention mask
        Returns:
            output: (batch, seq, d_model)
        """
        # ============================================
        # Pre-LN 구현:
        # 1. attn_out = attention(norm1(x), norm1(x), norm1(x))
        # 2. x = x + dropout(attn_out)
        # 3. ffn_out = ffn(norm2(x))
        # 4. x = x + ffn_out
        # ============================================
        
        pass  # 이 줄을 지우고 구현

In [ ]:
# ============ 테스트 ============
def test_encoder_block():
    batch, seq, d_model, num_heads = 2, 10, 64, 8
    
    encoder = EncoderBlock(d_model, num_heads)
    x = torch.randn(batch, seq, d_model)
    
    output = encoder(x)
    
    assert output.shape == x.shape, f"Shape 오류: {output.shape}"
    print("✅ Encoder Block 테스트 통과!")
    print(f"   Input: {x.shape} → Output: {output.shape}")

test_encoder_block()

---
## 2. Transformer Encoder (여러 블록 쌓기)

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, d_ff=None, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([
            EncoderBlock(d_model, num_heads, d_ff, dropout)
            for _ in range(num_layers)
        ])
        self.norm = nn.LayerNorm(d_model)  # 마지막 정규화
    
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

# 테스트
encoder = TransformerEncoder(d_model=64, num_heads=8, num_layers=6)
x = torch.randn(2, 10, 64)
output = encoder(x)
print(f"6-layer Encoder: {x.shape} → {output.shape}")
print(f"Total parameters: {sum(p.numel() for p in encoder.parameters()):,}")

---
## 정답

<details>
<summary>👉 클릭하여 정답 코드 보기</summary>

```python
class EncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff=None, dropout=0.1):
        super().__init__()
        
        self.attention = MultiHeadAttention(d_model, num_heads, dropout)
        self.ffn = PositionWiseFeedForward(d_model, d_ff, dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, mask=None):
        # Pre-LN: LayerNorm first
        normed = self.norm1(x)
        attn_out, _ = self.attention(normed, normed, normed, mask)
        x = x + self.dropout(attn_out)
        
        normed = self.norm2(x)
        ffn_out = self.ffn(normed)
        x = x + ffn_out
        
        return x
```

</details>